In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

class Content:
    def __init__(self, title, body):
        self.url = url
        self.title = title
        self.body = body
        
    def print(self):
        print(f'TITLE: {self.title}')
        print(f'URL: {self.url}')
        print(f'BODY: {self.body}')
        
def scrapeCNN(url):
    bs = BeautifulSoup(urlopen(url))
    title = bs.find('h1').text
    body = bs.find('div', {'class': 'article_content'}).text
    print('body: ')
    print(body)
    return Content(url, title, body)

def scrapeBrookings(url):
    bs = BeautifulSoup(urlopen(url))
    title = bs.find('h1').text
    body = bs.find('div', {'class': 'post-body'}).text
    return Content(url, title, body)

url = 'https://www.brookings.edu/research/robotic-rulemaking'
content = scrapeBrookings(url)
content.print()

url = 'https://www.cnn.com/2023/04/03/investing/\
dogecoin-elon-musk-twitter/index.html'
content = scrapeCNN(url)
content.print()

SyntaxError: unterminated string literal (detected at line 33) (853141281.py, line 33)

In [ ]:
class Content:
    """
    Common base class for all articles or pages
    """
    
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
        
    def print(self):
        """
        Flexible printing function to control output
        """
        print('URL: {}'.format(self.url))
        print('TITLE: {}'.format(self.title))
        print('BODY:\n{}'.format(self.body))
        
class Website:
    """
    Contains information about website structure.
    """
    def __init__(self, name, url, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag

In [ ]:
class Crawler:
    
    def getPage(url):
        try:
            html = urlopen(url)
        except Exception:
            return None
        return BeautifulSoup(html, 'html.parser')
    
    def safeGet(bs, selector):
        """
        Utility function used to get content string from a Beautiful Soup 
        object and a selector. Returns an empty string if no object is found
        for the given selector.
        """
        selectedElems = bs.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.join([elem.get_text() for elem in selectedElems])
        return ''
    
    def getContent(website, path):
        """
        Pull content from a given URL
        """
        url = website.url+path
        bs = Crawler.getPage(url)
        if bs is not None:
            title = Crawler.safeGet(bs.website.titleTag)
            body = Crawler.safeGet(bs.website.bodyTag)
            return Content(url, title, body)
        return Content(url, '', '')
    
    siteData = [
        ['O\'Reilly', 'https://www.oreilly.com', 'h1', 'div.title-description'],
        ['Reuters', 'https://www.reuters.com', 'h1', 'div.ArticleBodyWrapper'],
        ['Brookings', 'https://www.brookings.edu', 'h1', 'div.post-body'],
        ['CNN', 'https://www.cnn.com', 'h1', 'div.article_content']
    ]
    websites = []
    for name, url, title, body in siteData:
        websites.append(Website(name, url, title, body))
        
    Crawler.getContent(
        websites[0],
        '/library/view/web-scraping-with/9781491910283'
        ).print()
    Crawler.getContent(
        websites[1],
        '/article/us-usa-epa-pruitt-idUSKBN19W2D0'
        ).print()
    Crawler.getContent(
        websites[2],
        '/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/'
        ).print()
    Crawler.getContent(
        websites[3],
        '/2023/04/03/investing/dogecoin-elon-musk-twitter/index.html'
        ).print()
    

In [ ]:
class Content:
    """
    Common base class for all articles or pages
    """
    
    def __init__(self, topic, url, title, body):
        self.topic = topic
        self.url = url
        self.title = title
        self.body = body
        
    def print(self):
        """
        Flexible printing function to control output
        """
        print('New article found for topic: {}'.format(self.topic))
        print('URL: {}'.format(self.url))
        print('TITLE: {}'.format(self.title))
        print('BODY:\n{}'.format(self.body))
        
class Website:
    """
    Contains information about website structure.
    """
    def __init__(self, name, url, searchUrl, resultListing,
        resultUrl, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.searchUrl = searchUrl
        self.resultListing = resultListing
        self.resultUrl = resultUrl
        self.absoluteUrl = absoluteUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag
        
class Crawler:
    
    def __init__(self, website):
        self.site = website
        self.found = {}
        
    def getContent(self, topic, url):
        """
        Pull content from a given URL
        """
       # url = website.url+path
        bs = Crawler.getPage(url)
        if bs is not None:
            title = Crawler.safeGet(bs, self.site.titleTag)
            body = Crawler.safeGet(bs, self.site.bodyTag)
            return Content(topic, url, title, body)
        return Content(topic, url, '', '')
    
    def search(self, topic):
        """
        Searches a given site for a given topic and records all pages found.
        """
        bs = Crawler.getPage(self.site.searchUrl + topic)
        searchResults = bs.select(self.site.resultListing)
        for result in searchResults:
            url = result.select(self.site.resultUrl)[0].attrs['href']
            # Is this a relative or absolute URL?
            url = url if self.site.absoluteUrl else self.site.url + url
            if url not in self.found:
                self.found[url] = self.getContent(topic, url)
            self.found[url].print()
            
siteData = [
    ['Reuters', 'http://www.reuters.com', 
    'https://www.reuters.com/search/news?blob=',
    'div.search-result-indiv', 'h3.search-result-title a',
    False, 'h1', 'div.ArticleBodyWrapper'],
    ['Brookings', 'http://www.brookings.edu',
    'https://www.brookings.edu/search/?s=',
    'div.article-info', 'h4.title a', True, 'h1', 'div.core-block']
]
sites = []
for name, url, search, rListing, rUrl, absUrl, tt, bt in siteData:
    sites.append(Website(name, url, search, rListing, rUrl, absUrl, tt, bt))
    
crawlers = [Crawler(site) for site in sites]
topics = ['python', 'data%20science']

for topic in topics:
    for crawler in crawlers:
        crawler.search(topic)